# Parallel Programming

## Ejecuta tus programas en paralelo!

#### Fuentes utilizadas para este tutorial:
- [What is Parallel Programming and Multithreaded Programming?](https://www.perforce.com/blog/qac/multithreading-parallel-programming-c-cpp#:~:text=Parallel%20programming%20is%20the%20process,to%20Multithreading%20and%20Multithreaded%20Applications.)
- [Paralelism class from codeastro](https://github.com/semaphoreP/codeastro/blob/main/Day4/parallelism.ipynb)
- [Python concurrency and parallelism explained](https://www.infoworld.com/article/3632284/python-concurrency-and-parallelism-explained.html)

## Que es la programacion en paralelo?

La programacion en paralelo es el proceso de dividir parte del programa (o problema!) en pequeños segmentos que puedan ejecutarse **al mismo tiempo**. Esto es util cuando hay tareas que son independientes entre si, como por ejemplo tener que procesar diferentes imagenes o si es que tenemos que evaluar diferentes modelos que expliquen un fenomeno.

## Por que querriamos usar paralelismo?

Si bien las CPU cada vez son mejores, la realidad es que ya hemos alcanzado la velocidad maxima a la que estos pueden procesar informacion, por lo tanto la manera en que podemos exprimir todo el rendimiendo de las CPUs modernas es a traves del paralelismo.

Por otro lado es comun ver que existen procesos dentro de los programas que podrian ejecutarse en paralelo, asi acelerando el codigo en general, y quien no quiere que su codigo sea rapido? :P

## Como implementar paralelismo?

Existen muchos tipos de paralelismo que podemos implementar, cada uno con sus ventajas y complicaciones, sin embargo, para este tutorial veremos como implementar paralelismo en Python principalmente con `multiprocessing`, una libreria que nos permite ejecutar diferentes `procesos` en paralelo distribuido en diferentes nucleos de la CPU.

## Los mas basico de lo basico

Volvamos un poco al principio. El primer ejemplo que mencionamos fue procesar diferentes imagenes a la vez... Quizas no tenemos que paralelizar nuestro codigo para lograr esto, sino que podemos hacer algo mas sencillo... Como por ejemplo ejecutar el codigo varias veces al mismo tiempo en diferentes imagenes!

Ejecutar multiples instancias de Python es una manera **valida y viable** de paralelizar un proceso, aunque puede que no sea muy sofisticado. Esto es particularmente util en problemas sencillos ya que las herramientas de paralelizacion nativas de Python no son las mejores (gracias a la forma en que Python esta escrita!)

Hacer esto es relativamente sencillo, puedes escribir un script de `bash`, ejecutarlos a mano si no son demasiadas alternativas, o bien utilizar un script en Python para inicializar procesos de Python en paralelo!

## Que tan paralelo puedo ir?

Hay dos importantes limitaciones cuando se trata de paralelizar un programa:
1. El numero de nucleos al cual tienes acceso
2. La ganancia en velocidad vs el costo de iniciar nuevos procesos

El primer punto es facil de evaluar, y no es necesario tener que buscar la marca y modelo de tu CPU y luego googlear las especificaciones ni nada del estilo. Es mas, podemos buscar cuantos *cores* o *threads* tenemos desde Python! Veamos en seguida como hacer esto:

In [1]:
import multiprocessing

multiprocessing.cpu_count()

16

Utilizando la funcion `cpu_count()` del modulo `multiprocessing` sabemos cuantos threads o procesos podemos inicializar en paralelo antes que la CPU empiece a tropezar consigo misma :P

El segundo punto es mas complicado de evaluar, pero la idea es la siguiente: cuando uno paraleliza un codigo, en algun momento deben inicializarse nuevos procesos. A veces estos procesos pueden compartir todas las variables y direcciones de memoria, etc. A esto se le llama *multithreading*, y si bien Python es capaz de llevar a cabo multithreading, esto no siempre es recomendable gracias a la estructura interna de Python. La alternativa ya la conocemos y es `multiprocessing`, en donde cada proceso es independiente entre si y toda la informacion que se deberia compartir entre ellas en realidad se duplica y se le asigna a cada proceso, ademas los resultados deben serializarse para volver al programa principal cuando la ejecucion haya finalizado.

Como resultado de esto, hay un costo significativo en utilizar `multiprocessing` y puede ocurrir que al paralelizar el codigo, este termine siendo aun mas lento que si se hubiese ejecutado normalmente!

Otra consecuencia de paralelizar el codigo es que, si bien obtenemos mayor velocidad al utilizar un mayor numero de *cores*, esto no crece infinitamente y eventualmente no vamos a ganar mas velocidad. Es mas, puede ocurrir todo lo contrario!

Por estos motivos es importante evaluar cual es el numero ideal de *cores* para tu codigo, asi en caso que estemos utilizando un servidor compartido no vamos a utilizar el maximo de threads a ciegas y perjudicar el trabajo de nuestros colegas!

## Multiprocessing

Veremos algunos ejemplos de como podemos utilizar `multiprocessing` para nuestros proyectos. El archivo `parallel_funpy` tiene una funcion llamada `funcion_paralelizada` que utilizaremos como ejemplo. Lo tenemos en un archivo separado para que funcione en `jupyter` :)

In [2]:
from multiprocessing import Process, Queue
import numpy as np

razas = np.loadtxt('razas.txt', dtype=str, delimiter='\t')

In [3]:
from parallel_fun import funcion_paralelizada_process

output = Queue()

procesos = []
for raza in razas:
    proceso = Process(target=funcion_paralelizada_process, args=(raza, output))
    procesos.append(proceso)
    proceso.start()

for proceso in procesos:
    proceso.join()
    resultado = output.get(proceso)
    print(resultado)


La raza de este perro es Airedale terrier
La raza de este perro es affenpinscher
La raza de este perro es American Staffordshire terrier
La raza de este perro es Afghan hound
La raza de este perro es Akita
La raza de este perro es Alaskan Malamute
La raza de este perro es Australian terrier
La raza de este perro es Australian shepherd
La raza de este perro es basset hound
La raza de este perro es American water spaniel
La raza de este perro es Australian cattle dog
La raza de este perro es basenji
La raza de este perro es black and tan coonhound
La raza de este perro es bichon frise
La raza de este perro es beagle
La raza de este perro es Bedlington terrier
La raza de este perro es bouvier des Flandres
La raza de este perro es Bernese mountain dog
La raza de este perro es Brussels griffon
La raza de este perro es bearded collie
La raza de este perro es bulldog
La raza de este perro es border terrier
La raza de este perro es boxer
La raza de este perro es Boston terrier
La raza de este 

Si revisan el archivo `razas.txt` veran que el programa no ha imprimido las razas en el mismo orden. Este es un punto importante a considerar si van a utilizar `multiprocessing` de esta manera!

De todas maneras hay mejores formas de lograr lo que hicimos en el codigo anterior utilizando la clase `Pool` de `multiprocessing`!
Veamos como replicar lo anterior con `Pool`


In [4]:
from multiprocessing import Pool

from parallel_fun import funcion_paralelizada_pool

with Pool(4) as p:
    outs = p.map(funcion_paralelizada_pool, razas)
    print('\n'.join(outs))

La raza de este perro es affenpinscher
La raza de este perro es Afghan hound
La raza de este perro es Airedale terrier
La raza de este perro es Akita
La raza de este perro es Alaskan Malamute
La raza de este perro es American Staffordshire terrier
La raza de este perro es American water spaniel
La raza de este perro es Australian cattle dog
La raza de este perro es Australian shepherd
La raza de este perro es Australian terrier
La raza de este perro es basenji
La raza de este perro es basset hound
La raza de este perro es beagle
La raza de este perro es bearded collie
La raza de este perro es Bedlington terrier
La raza de este perro es Bernese mountain dog
La raza de este perro es bichon frise
La raza de este perro es black and tan coonhound
La raza de este perro es bloodhound
La raza de este perro es border collie
La raza de este perro es border terrier
La raza de este perro es borzoi
La raza de este perro es Boston terrier
La raza de este perro es bouvier des Flandres
La raza de este

En el codigo anterior, creamos un grupo de trabajadores distribuidos en 4 cores con `Pool`, y le dijimos que distribuyera la ejecucion de la funcion `funcion_paralelizada_pool` en ese conjunto a traves de la funcion `map`. Utilizar el *context manager* `with` es importante porque de esa manera no tenemos que cerrar el conjunto con `p.close()` manualmente!!

Utilizar `Pool` junto con `map` es una manera comun de paralelizar codigo cuando **nos interesa el orden** y **cuando lo que queramos paralelizar no sea muy pesado en memoria**, en caso de que tengamos un iterable gigante (por ejemplo si quisieramos hacer un histograma con miles de millones de datos), sera necesario utilizar `imap` en vez de `map`. `imap` conserva el orden del input inicial, si el orden no nos es relevante, entonces podemos utilizar `imap_unordered`.

In [5]:
with Pool(4) as p:
    outs = p.imap_unordered(funcion_paralelizada_pool, razas)
    print('\n'.join(outs))

La raza de este perro es Afghan hound
La raza de este perro es Alaskan Malamute
La raza de este perro es American Staffordshire terrier
La raza de este perro es American water spaniel
La raza de este perro es Australian cattle dog
La raza de este perro es Australian shepherd
La raza de este perro es Australian terrier
La raza de este perro es basenji
La raza de este perro es basset hound
La raza de este perro es beagle
La raza de este perro es bearded collie
La raza de este perro es Bedlington terrier
La raza de este perro es Bernese mountain dog
La raza de este perro es bichon frise
La raza de este perro es black and tan coonhound
La raza de este perro es bloodhound
La raza de este perro es border collie
La raza de este perro es border terrier
La raza de este perro es borzoi
La raza de este perro es Boston terrier
La raza de este perro es bouvier des Flandres
La raza de este perro es boxer
La raza de este perro es briard
La raza de este perro es Brittany
La raza de este perro es Bruss

## Palabra final

Hay mucho mas de que hablar en este tema, y esto es solo un ejemplo basico para que ustedes puedan implementar en sus propios proyectos.

Existe `GNU Parallel` por ejemplo, en sistemas basados en UNIX, algunas librerias pueden ofrecer paralelismo a traves de `MPI`. Ahora es tarea de ustedes ahondar en esto y ver que necesitan para acelerar sus codigos!

# Actividad: Procesamiento de imagenes

Ejecuten el codigo siguiente para generar datos falsos y luego apliquen la funcion para procesar imagenes en cada imagen falsa, guardando los resultados. Que tan rapido pueden procesar la data cuando cada imagen es de 1000x1000? Cuanto es la ganancia en rapidez aumentando desde 1 core a varios? Es una ganancia lineal?

Pueden obtener el timpo de ejecucion utilizando el modulo `time`. Por ejemplo:

```python
import time

inicio = time.time()
# Algun proceso pesado
delta_t = time.time() - inicio
print(delta_t)
```
**Deben tener instalado astropy para este ejercicio**

In [ ]:
import astropy.io.fits as fits

# note, this cell might take a minute to run.

def generate_fake_data(filename, dims):
    """
    Generates a fake dataframe with random numbers
    
    Args:
        filename (str): file to save the data to
        dims (tuple): (Ny, Nx) pair that species the size of the y and x dimensions
    """
    # some complicated random image generation. Feel free to ignore.
    # coordinate system in fourier spae
    u,v = np.meshgrid(np.fft.fftfreq(dims[1]), np.fft.fftfreq(dims[0]))
    phases = np.random.uniform(0, 2*np.pi, u.shape)
    rho = np.sqrt((u*dims[1])**2 + (v*dims[0])**2)
    # suppress high frequency by a squared exponential
    spectrum = np.exp(-rho**2/(np.max(rho)/50)**2)  * np.exp(1j * phases)
    filtered = np.real(np.fft.ifft2(spectrum))

    fits.writeto(filename, filtered, overwrite=True)


# generate fake data (can choose to save it somethere else if you want)
fileformat = os.path.join("./", "fake_{0}x{1}_{2}.fits")

ny = 1000
nx = 1000
for i in range(25):
    filename = fileformat.format(ny, nx, i)
    generate_fake_data(filename, (ny, nx) )

In [ ]:
def process_image(frame, filtersize=50):
    """
    Run a high-pass filter on the data. 
    Remove the low spatial frequency (i.e., smooth features) in the image

    Args:
        frame (np.array): a 2-D image to be processed
        fitersize (int): the size of the filter. Features smaller than the filtersize will be preserved

    Returns:
        processed_frame (np.array): a 2-D image after processing
    """
    # run a median filter to smooth the image
    frame_smooth = ndimage.median_filter(frame, filtersize)

    processed_frame = frame - frame_smooth

    return processed_frame

# an example of running this on one image
with fits.open(fileformat.format(ny, nx, 0)) as hdulist:
    data = hdulist[0].data


    filt_data = process_image(data)

    fig = plt.figure(figsize=(6,3))
    ax1 = fig.add_subplot(121)
    ax1.imshow(data, cmap="inferno")
    ax1.set_title("Original")
    ax2 = fig.add_subplot(122)
    ax2.imshow(filt_data, cmap="inferno")
    ax2.set_title("Filtered")

#### Activity:
# write and time some code that runs this on all 25 images in parallel. How does the performance increase
# as you increase the number of processes you use?
# we recommend you use multiprocessing pool for this task